In [164]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline as PipeIMB
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.utils import class_weight
from imblearn.under_sampling import ClusterCentroids
from sklearn.multiclass import OneVsOneClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split


In [8]:
train = pd.DataFrame(np.load('../elrich/x_train_features.npy'))
y = np.load('../elrich/y_train_values.npy')
train

,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,64,65,66,67
0,243.597015,98.0,256.0,22.185823,251.129032,218.0,504.0,33.234496,243.126984,125.0,...,124.185529,9.952609,0.011488,-37.602443,37.320107,7.029856,0.811568,0.326647,0.853283,0.074444
1,235.606061,208.0,257.0,12.761066,234.433333,164.0,274.0,20.196837,232.206897,154.0,...,85.586597,3.883948,-0.137922,-3.205128,2.411407,1.493474,0.784593,0.693250,0.856564,0.043021
2,269.551724,251.0,297.0,10.578398,271.153846,201.0,349.0,32.314193,271.040000,252.0,...,70.603353,2.230728,0.125242,-2.186039,2.006572,1.117426,0.898105,0.836565,0.989880,0.035847
3,123.376812,101.0,204.0,21.830773,857.250000,144.0,2813.0,834.007606,122.553846,97.0,...,170.026161,11.912734,-0.108934,-26.143791,25.886388,10.429553,0.409855,0.336628,0.679922,0.072426
4,179.697674,66.0,221.0,23.296406,205.200000,143.0,490.0,74.717296,179.923077,75.0,...,108.436358,6.004780,-0.052222,-10.608886,11.026965,3.222062,0.613908,0.503271,0.956549,0.072437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5108,217.043478,78.0,311.0,61.584497,250.166667,94.0,515.0,81.204988,218.315789,99.0,...,139.401010,20.812725,0.458240,-27.819755,33.615268,16.535187,0.757796,0.353268,1.036475,0.187219
5109,259.000000,252.0,275.0,3.741657,258.222222,205.0,315.0,30.910065,258.038462,184.0,...,70.868347,0.785999,-0.099029,-1.437632,0.532615,0.469217,0.863229,0.839899,0.916556,0.012684
5110,236.656250,201.0,298.0,29.022200,234.965517,196.0,303.0,29.998256,235.964286,185.0,...,89.404463,8.827483,-0.552520,-4.836081,3.437164,1.928052,0.791621,0.669916,0.993208,0.096930
5111,254.500000,238.0,269.0,8.257572,254.655172,234.0,273.0,10.117502,254.321429,171.0,...,74.324551,1.370485,0.086599,-2.309321,1.588037,1.225818,0.847321,0.793241,0.896562,0.027480


In [15]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')
x_data = imp.fit_transform(train)
y_data = y

In [178]:
scorer = make_scorer(f1_score, average='micro')
def cv(model):
    return cross_val_score(model, x_data, y_data, cv=8, scoring=scorer, verbose=2, n_jobs=-1)

In [27]:
gb_model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=3)
gb_scores = cv(gb_model)
#0.8028555375489237

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   38.6s remaining:   16.5s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   55.3s finished


In [59]:
y_s = pd.DataFrame({'y':y})
y_s.groupby('y').size()

y
0    3030
1     443
2    1470
3     170
dtype: int64

In [104]:
cl_w = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_data),
                                                 y_data)
class_weight_dict = dict(enumerate(cl_w))
class_weight_dict

{0: 0.4218646864686469,
 1: 2.8854401805869077,
 2: 0.8695578231292517,
 3: 7.519117647058824}

In [160]:
sample_weight = np.array([class_weight_dict[i] for i in y_data])

In [195]:
pipe_steps = [('XGB', xgb.XGBClassifier(n_estimators=200, objective = 'multi:softmax', num_class=4))]

check_params = {
    'XGB__max_depth': [2, 3],
    'XGB__learning_rate': [0.1, 0.01],
    'XGB__min_child_weight' : [1, 2, 3]
    #'SVM__gamma':np.logspace(-9, -3, 6)
}

pipeline = Pipeline(pipe_steps)

In [ ]:
# xgb_scores = cv(pipeline)
# np.mean(xgb_scores)
#0.803248073630137
create_grid=GridSearchCV(pipeline, check_params,cv=8, scoring=scorer, n_jobs=-1, verbose=2)
create_grid.fit(x_data,y_data)
print("Best fit")
print(create_grid.best_params_)
print(create_grid.best_score_)

Fitting 8 folds for each of 12 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  6.2min finished


In [186]:
test.shape

(3411, 68)

In [189]:
x_data

array([[2.43597015e+02, 9.80000000e+01, 2.56000000e+02, ...,
        3.26647287e-01, 8.53282708e-01, 7.44435911e-02],
       [2.35606061e+02, 2.08000000e+02, 2.57000000e+02, ...,
        6.93249870e-01, 8.56563541e-01, 4.30214544e-02],
       [2.69551724e+02, 2.51000000e+02, 2.97000000e+02, ...,
        8.36565302e-01, 9.89880058e-01, 3.58467832e-02],
       ...,
       [2.36656250e+02, 2.01000000e+02, 2.98000000e+02, ...,
        6.69915755e-01, 9.93208433e-01, 9.69298395e-02],
       [2.54500000e+02, 2.38000000e+02, 2.69000000e+02, ...,
        7.93240956e-01, 8.96562257e-01, 2.74803847e-02],
       [1.77325581e+02, 1.60000000e+02, 2.03000000e+02, ...,
        5.33266700e-01, 6.76582126e-01, 3.69300891e-02]])

In [190]:
test = pd.DataFrame(np.load('../elrich/x_test_data.npy'))
clf = xgb.XGBClassifier(learning_rate = 0.1, max_depth = 3, n_estimators=200, objective = 'multi:softmax', num_class=4)
clf.fit(x_data, y_data)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, num_class=4, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=1, verbosity=1)

In [191]:
pred = clf.predict(test.values)

In [192]:
test_y = pred
file_test = "xgb_best"
output = pd.DataFrame({'id':[float(i) for i in range(0,len(test_y))],'y':test_y})
output.to_csv("{}.csv".format(file_test),index=False)

In [165]:
#x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3, stratify = y_data)

In [144]:
pipe_steps = [('CC',SMOTE('not majority')), ('scaler', StandardScaler()) ,
              ('SVM', svm.SVC(kernel = 'poly', gamma='scale', coef0=1, degree=4,
                                                                decision_function_shape='ovo',
                                                                cache_size=3000, random_state = 42
                                                                ))]

check_params = {
    'SVM__degree': [2,3,4,5],
    'SVM__coef0': [0,1]
    #'SVM__gamma':np.logspace(-9, -3, 6)
}

pipeline = PipeIMB(pipe_steps)

In [145]:
svm_scores = cv(pipeline)
np.mean(svm_scores)
# create_grid=GridSearchCV(pipeline, check_params,cv=10, scoring=scorer, n_jobs=-1, verbose=2)
# create_grid.fit(x_data,y_data)
# print("Best fit")
# print(create_grid.best_params_)
# print(create_grid.best_score_)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   54.3s remaining:   23.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.2min finished


0.7817270058708414

In [ ]:
#0.775079118762231 degree 4